# Project 6

## Assignment Scenario

You are a data scientist in a call center. You are given a tab-delimited bilingual sentence pair contained in the file, deu-eng.txt Download deu-eng.txt.

The dataset has the following format:

English + TAB + German + TAB + Attribution

You are asked to develop a sequence to sequence model using RNN to translate English to German using the attached Jupyter Notebook and writing a script in Python, and running all the cells. You only need to submit a Jupyter Notebook.

**Steps and Questions**

1. Load the data, deu-eng.txt Download deu-eng.txt, into memory and clean and prepare the data.

2. Develop an encoder for the model.

3. Develop a decoder for the model.

4. Build the sequence to sequence model.

5. Train the model.

6. Evaluate the model and check the model fit.

7. Run the inference.

## Load the Necessary Libraries 

In [1]:
# Data handling and preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Tokenization and padding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model building
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Model training
from tensorflow.keras.callbacks import EarlyStopping

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1: Load, Clean, and Prepare the Data

In [2]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Set file path
data_path = "/Users/matthewmoore/Downloads/deu-eng.txt"

# Initialize variables
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Read and process the data
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[:10000]:  # Limit the number of samples to 10,000
    if "\t" not in line:
        continue
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

# Sort the characters
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# Define vocabulary sizes and sequence lengths
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print(f"Number of samples: {len(input_texts)}")
print(f"Number of unique input tokens: {num_encoder_tokens}")
print(f"Number of unique output tokens: {num_decoder_tokens}")
print(f"Max sequence length for inputs: {max_encoder_seq_length}")
print(f"Max sequence length for outputs: {max_decoder_seq_length}")


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 85
Max sequence length for inputs: 15
Max sequence length for outputs: 45


## Step 2-3: Develop an Encoder and Decoder for the Model

In [3]:
# Create dictionaries for character-to-index and index-to-character mapping
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
target_char_index = {char: i for i, char in enumerate(sorted(list(target_characters)))}
reverse_target_char_index = {i: char for char, i in target_char_index.items()}

# Initialize data arrays
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

# Populate the data arrays
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


## Step 4: Develop the Sequence to Sequence

In [4]:
# Define dimensions
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Combined model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Step 5: Train the Model

In [5]:
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping

# Compile the model with Adam optimizer
training_model.compile(optimizer=RMSprop(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

# Early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train the model
history = training_model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stopping],
)

Epoch 1/50
125/125 [==============================] - 15s 103ms/step - loss: 1.4303 - accuracy: 0.0525 - val_loss: 1.4847 - val_accuracy: 0.0740
Epoch 2/50
125/125 [==============================] - 12s 99ms/step - loss: 1.2982 - accuracy: 0.0616 - val_loss: 1.4456 - val_accuracy: 0.0834
Epoch 3/50
125/125 [==============================] - 12s 99ms/step - loss: 1.2741 - accuracy: 0.0650 - val_loss: 1.4355 - val_accuracy: 0.0880
Epoch 4/50
125/125 [==============================] - 12s 99ms/step - loss: 1.2524 - accuracy: 0.0691 - val_loss: 1.3949 - val_accuracy: 0.0861
Epoch 5/50
125/125 [==============================] - 12s 99ms/step - loss: 1.2331 - accuracy: 0.0752 - val_loss: 1.3996 - val_accuracy: 0.1047
Epoch 6/50
125/125 [==============================] - 12s 99ms/step - loss: 1.2154 - accuracy: 0.0832 - val_loss: 1.3473 - val_accuracy: 0.1106
Epoch 7/50
125/125 [==============================] - 12s 99ms/step - loss: 1.1989 - accuracy: 0.0896 - val_loss: 1.3340 - val_accuracy

In [6]:
# Save the entire model for future inference
training_model.save("seq2seq_model.h5")

/Users/matthewmoore/opt/anaconda3/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Step 6: Evaluate the Model and Check Fit

In [7]:
# Evaluate the model on validation data
val_loss, val_accuracy = training_model.evaluate(
    [encoder_input_data, decoder_input_data], decoder_target_data, verbose=1
)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


313/313 [==============================] - 7s 22ms/step - loss: 0.9191 - accuracy: 0.1813
Validation Loss: 0.9191
Validation Accuracy: 0.1813


## Step 7: Run the Inference

In [8]:
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [9]:
def decode_sequence(input_seq):
    # Encode the input sequence to get the context vector
    states_value = encoder_model.predict(input_seq)
    
    # Generate the start-of-sequence token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    # Initialize the decoded sentence
    decoded_sentence = ""
    stop_condition = False

    while not stop_condition:
        # Predict the next token
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Check for stop condition
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence with the sampled token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update the states
        states_value = [h, c]

    return decoded_sentence



In [10]:
# Generate and decode predictions for 10 sequences
for seq_index in range(10):
    # Take one sequence from the test or validation set for decoding
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    
    # Use the decoder to predict the sequence
    decoded_sentence = decode_sequence(input_seq)
    
    print("-" * 50)
    print(f"Input Sentence: {input_texts[seq_index]}")
    print(f"Decoded Sentence: {decoded_sentence}")
    print(f"Target Sentence: {target_texts[seq_index]}")

1/1 [==============================] - 0s 16ms/step
--------------------------------------------------
Input Sentence: Go.
Decoded Sentence: Gen eeneneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
Target Sentence: 	Geh.

1/1 [==============================] - 0s 16ms/step
--------------------------------------------------
Input Sentence: Hi.
Decoded Sentence: Alleeeneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
Target Sentence: 	Hallo!

1/1 [==============================] - 0s 16ms/step
--------------------------------------------------
Input Sentence: Hi.
Decoded Sentence: Alleeeneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
Target Sentence: 	Grüß Gott!

1/1 [==============================] - 0s 15ms/step
--------------------------------------------------
Input Sentence: Run!
Decoded Sentence: Alleeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
Target Sentence: 	Lauf!

1/1 [==============================] - 0s 15ms/step
--------------------------------------------------
Input Sentence: Run.
Decoded Sentence: 